In [3]:
import os
import pandas as pd
import re

def masterfile():
    # Paths
    folderPath = "FOLDER"  # Update this
    masterPath = "Master datasheet.xlsx"  # Update this
    outputPath = "master/master.xlsx"  # Update this

    def generateCol(n):
        """ Generate Excel-style column names (A, B, C... AA, AB, etc.). """
        names = []
        for i in range(n):
            col_name = ''
            while i >= 0:
                col_name = chr(i % 26 + 65) + col_name
                i = i // 26 - 1
            names.append(col_name)
        return names

    # Read the master template
    master = pd.read_excel(masterPath)
    masterColnames = master.columns  # Preserve original column names
    n = len(master.columns)
    Colnames = generateCol(n)
    master.columns = Colnames  # Use temporary column names for processing

    # Loop through all folders in the directory
    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)
        if not os.path.isdir(paths):  # Skip files, only process directories
            continue
        
        for filename in os.listdir(paths):
            filePath = os.path.join(paths, filename)

            try:
                # Read the column mapping sheet (handle missing sheets)
                try:
                    Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                except ValueError:
                    print(f"Warning: Sheet '{foldername}' not found in master template. Skipping.")
                    continue

                if not {'From', 'To'}.issubset(Col_directions.columns):
                    print(f"Error: Mapping sheet {foldername} must contain 'From' and 'To' columns.")
                    continue

                # Read the file to process
                fileRead = pd.read_excel(filePath)
                temp_data = []  # Temporary storage for processed rows

                # Breaking measurement columns into M1, M2, M3
                if 'Measurement' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].str.split(' x ', expand=True)
                    fileRead[['m1', 'm2', 'm3']] = fileRead[['m1', 'm2', 'm3']].apply(pd.to_numeric, errors='coerce')

                def extract_dimensions(measurement):
                    match = re.match(r"(\d+\.\d+)-(\d+\.\d+)\*(\d+\.\d+)", str(measurement))
                    return match.groups() if match else (None, None, None)
           
                if 'Diameter' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Diameter'].apply(lambda x: pd.Series(extract_dimensions(x)))
             
                # Assign dynamic column names to the current file
                n = len(fileRead.columns)
                Colnames = generateCol(n)
                fileRead.columns = Colnames

                # Process rows based on column mapping
                for _, row in fileRead.iterrows():
                    # if pd.isna(row['A']):  # Stop processing if empty row
                    #     break

                    new_row = {'A': filename}  # Track file name for reference

                    for _, movement_row in Col_directions.iterrows():
                        from_col = movement_row['From']
                        to_col = movement_row['To']

                        if from_col in row.index and to_col in master.columns:
                            new_row[to_col] = row[from_col]

                    temp_data.append(new_row)  # Add processed row

                # Append processed data to master
                temp_df = pd.DataFrame(temp_data)
                master = pd.concat([master, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
            
    # Function to calculate 'R' column
    def calculate_R(row):
        if 'P' in row and 'Q' in row and pd.notna(row['P']) and pd.notna(row['Q']):
            return ((100 - row['Q']) / 100) * row['P'] if row['Q'] > 0 else ((100 + row['Q']) / 100) * row['P']
        return None

    # Function to calculate 'S' column
    def calculate_S(row):
        if 'R' in row and 'C' in row and pd.notna(row['R']) and pd.notna(row['C']):
            return round(row['R'] * row['C'])  # Perform calculation
        return None

    # Apply calculations
    if 'P' in master.columns and 'Q' in master.columns:
        master['R'] = master.apply(calculate_R, axis=1)
    if 'R' in master.columns and 'C' in master.columns:
        master['S'] = master.apply(calculate_S, axis=1)

    # Clean column 'I'
    if 'I' in master.columns:
        master['I'] = master['I'].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)

    # Restore original column names
    master.columns = masterColnames

    # Save the final master file
    os.makedirs(os.path.dirname(outputPath), exist_ok=True)  # Ensure directory exists
    master.to_excel(outputPath, index=False)
    print(f"Master file updated and saved at {outputPath}.")



In [4]:
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import column_index_from_string

def hyperlink():
    folderPath = "FOLDER"
    masterPath = "Master datasheet.xlsx"
    outputPath = "master/master.xlsx"
    targetCol = column_index_from_string('X')  # Column A for data
    # Column B for filenames
    ReferenceCol = column_index_from_string('AD')
    fileCol = column_index_from_string('AC')
    rowCount = 2

    if os.path.exists("master\\master.xlsx"):
        target = load_workbook("master\\master.xlsx")
        targetSheet = target.active
     # Add headers

    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)

        for filename in os.listdir(paths):
            try:
                filePath = os.path.join(paths, filename)
                Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                source = load_workbook(filePath)
                sourceSheet = source.active

                sourceCol = column_index_from_string(Col_directions['Hfrom'][0])
                RefCol = column_index_from_string(Col_directions['StkRf'][0])

                for count, row in enumerate(sourceSheet.iter_rows(min_row=2), start=2):
                    cell = row[sourceCol - 1]
                    RefCell = row[RefCol-1]
                    targetCell = targetSheet.cell(row=rowCount, column=targetCol)
                    targetCell.value = cell.value
                    
                    

                    # Store filename in column B
                    

                    if cell.hyperlink:
                        targetCell.hyperlink = cell.hyperlink
                        targetCell.style = "Hyperlink"

                        rowCount += 1

                print(f"Processed {filename} successfully")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    target.save(outputPath)
    print("Done")






In [31]:
import pandas as pd
import psycopg2
from openpyxl import load_workbook

def database_transfer():
    conn = psycopg2.connect(
        user= "postgres",
        host= "localhost",
        database= "Master_Compilation",
        password="123",
        port= 5432,
    )
    db = conn.cursor()
    file = "master\master.xlsx"
    wb = load_workbook(file, data_only=True)
    ws = wb.active
    df = pd.read_excel(file,engine="openpyxl")
    columns = list(df.columns)
    columns.append('Date')
    vedio_links=[]
    for row in ws.iter_rows(min_row=2,max_row=ws.max_row,min_col=df.columns.get_loc("Video link")+1,max_col=df.columns.get_loc("Video link")+1):
        cell = row[0]
        if cell.hyperlink:
            vedio_links.append(cell.hyperlink.target)
        else:
            vedio_links.append(None)
    df['Video link'] =  vedio_links
    numeric_columns = ["Size", "Ratio", "M1", "M2", "M3", "Depth", "Table",
                   "Rap rate", "Discount", "PPC", "Total"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    display(df)
    for index, row in df.iterrows():
        db.execute(''' insert into diamonds (company,shape,size,colour,clarity,cut,polish,symmetry,
                flour,ratio,m1,m2,m3,depth,tables,rap_rate,discount,ppc,total,total_in_rs,report_no,
                stockref,cert_link,video_link,binC,binS,comment,comment2) Values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',
                tuple(row))
    conn.commit()
    db.close()
    conn.close()
    print("successfully added")
    
        
database_transfer()

,Company,Shape,Size,Colour,Clarity,Cut,Polish,Symmetry,Flour,Ratio,...,Total,Total in Rs,Report no.,Stockref,Cert link,Video link,BinC,BinS,Comment,Comment 2
0,DDPLStock-21-Jan-2025-10-28-57.xlsx,PEAR,0.90,E +,VVS1 -,-,EX,VG,NONE,1.574,...,2967.0,NaN,*****48341,XSNBO004,GIA,https://assets.3dvirtualdiamond.com/mplayer/XS...,No,No,Feather,NaN
1,DDPLStock-21-Jan-2025-10-28-57.xlsx,PEAR,0.90,H -,VS1 -,-,EX,EX,NONE,1.672,...,2039.0,NaN,*****97575,XSNPU019,GIA,https://assets.3dvirtualdiamond.com/mplayer/XS...,No,No,"Feather, Cloud, Pinpoint",NaN
2,DDPLStock-21-Jan-2025-10-28-57.xlsx,PEAR,0.90,H -,VS1,-,VG,EX,NONE,1.623,...,2039.0,NaN,*****67387,XSNSN069,GIA,https://assets.3dvirtualdiamond.com/mplayer/XS...,No,No,Feather,NaN
3,DDPLStock-21-Jan-2025-10-28-57.xlsx,PEAR,0.74,I,IF,-,EX,VG,NONE,1.572,...,1292.0,NaN,*****01674,XSNRB075,GIA,https://assets.3dvirtualdiamond.com/mplayer/XS...,No,No,Minor Details of Polish,NaN
4,DDPLStock-21-Jan-2025-10-28-57.xlsx,PEAR,0.71,I -,VVS1,-,EX,VG,NONE,1.599,...,1170.0,NaN,*****01785,XSNRU033,GIA,https://assets.3dvirtualdiamond.com/mplayer/XS...,No,No,Pinpoint,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Lab_EXL_210125105200.xlsx,RN,0.76,D,VVS1,EX+,EX,EX,NaN,1.391,...,1768.0,NaN,NaN,101470131,GIA,https://d1s5m21q2l18ke.cloudfront.net/v360_mov...,NaN,NaN,Pinpoint,NaN
1067,Lab_EXL_210125105200.xlsx,RN,0.79,D,VVS1,EX+,EX,EX,NaN,1.441,...,1858.0,NaN,NaN,102280400,GIA,https://d1s5m21q2l18ke.cloudfront.net/v360_mov...,S,NaN,"Pinpoint, Needle, Feather",NaN
1068,Lab_EXL_210125105200.xlsx,RN,0.75,D,IF,EX+,EX,EX,NaN,1.385,...,1840.0,NaN,NaN,102060217,GIA,https://d1s5m21q2l18ke.cloudfront.net/v360_mov...,NaN,NaN,Minor Details of Polish,NaN
1069,Lab_EXL_210125105200.xlsx,PC,0.90,F,VVS1,EX,EX,EX,NaN,1.000,...,2256.0,NaN,NaN,103510490,GIA,https://d1s5m21q2l18ke.cloudfront.net/v360_mov...,NaN,NaN,"Internal Graining, Feather",NaN


successfully added


In [6]:
import threading

t1= threading.Thread(target=masterfile)
t2 = threading.Thread(target=hyperlink)

t1.start()
t1.join()
t2.start()
t2.join()

C:\Users\91965\AppData\Local\Temp\ipykernel_6600\2557374625.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master = pd.concat([master, temp_df], ignore_index=True)


Master file updated and saved at master/master.xlsx.
Processed DDPLStock-21-Jan-2025-10-28-57.xlsx successfully
Processed FINESTAR_21_01_2025_04_39_52487.xlsx successfully
Processed FINESTAR_21_01_2025_04_43_53IBT.xlsx successfully
Processed Jodhani-Brothers-Stock-21012025-10-15-54.xlsx successfully
Processed Lab_EXL_210125104511.xlsx successfully
Processed Lab_EXL_210125104820.xlsx successfully
Processed Lab_EXL_210125105200.xlsx successfully
Done
